In [ ]:
!pip install forex_python

импортируем библиотеки

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from datetime import datetime
from forex_python.converter import CurrencyRates
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import requests
from xml.etree import ElementTree as ET
import re
from collections import defaultdict


файлик не читался, поэтому глянули первые сточки


In [ ]:
with open('/content/it.csv', 'r') as f:
    for _ in range(5):
        print(f.readline())

﻿data_source;v_id;id;date_published;is_open;name;description;salary_from;salary_to;salary_currency;experience;employer_name;employer_type_label;employer_industry_id;employer_industry_name;employer_department_name;accept_kids;accept_handicapped;schedule_label;employment_label;country_id;country_name;region_id;region_name;region_district_id;region_district_name;role_id;role_name;skills;hard_skills;soft_skills

"superjob";"002c181d-02ff-bc9c-0a61-84f51501f72a";"ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57";9/10/2023 03:35:02 PM;1;"Инженер по пусконаладке станков / Сервисный инженер-наладчик / Инженер-механик (машиностроение)";"Наладка и проведение испытаний балансировочных станков перед  отправкой Заказчику. Проведение пуско-наладочных работ и ввод в эксплуатацию балансировочных станков на предприятиях Заказчика..\nОбразование высшее техническое, желательно факультет ""Машиностроение"" или ""Станкостроение"". Наличие практического опыта по  монтажу, испытаниям и  наладке  станков.  Владение ПК.  

косяк поняли, веселые разделения, ну теперь и прочсеть можно

In [ ]:
df = pd.read_csv('/content/it.csv', sep=';', engine='python')
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,country_name,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,9/10/2023 03:35:02 PM,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,Россия,347,Москва,1,Центральный,0,NaN,[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,9/9/2023 03:40:03 PM,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,Россия,578,Ханты-Мансийский АО - Югра,5,Уральский,0,NaN,[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,8/31/2023 01:20:03 PM,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,Россия,475,Санкт-Петербург,2,Северо-Западный,0,NaN,[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,3/18/2023 01:20:03 PM,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,Россия,347,Москва,1,Центральный,0,NaN,[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,7/13/2023 01:35:04 PM,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,Россия,347,Москва,1,Центральный,0,NaN,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,12/16/2022 11:54:22 AM,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,NaN,...,Россия,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме..."
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,12/26/2022 10:11:42 PM,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,Россия,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[]
171137,hh,fcc1a67f-a63b-4194-f288-aa7173e0830e,76596362,2/3/2023 02:10:00 PM,0,Кладовщица / Кладовщик,Обязанности: знание методов ведения и учета с...,40000,45000,RUR,...,Россия,464,Рязанская область,1,Центральный,52,Кладовщик,[],[],[]
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,4/27/2023 06:38:18 AM,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,Казахстан,26,Алматы,0,NaN,40,Другое,[],[],[]


выглядит многообещающе...
171140 строчек и 31 колонка, красота

In [ ]:
print(f"{df.duplicated().sum()}")

0


полных дубликатов не обнаружили, ну и ладно

приведем вакансии к единому формату
 приводим к нижнему регистру , удаляем спецсимволы , удаляем двойные пробелы , обрезаем пробелы по краям


In [ ]:
df['role_name'] = df['role_name'].fillna(df['name'])
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,country_name,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,9/10/2023 03:35:02 PM,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,Россия,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,9/9/2023 03:40:03 PM,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,Россия,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,8/31/2023 01:20:03 PM,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,Россия,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,3/18/2023 01:20:03 PM,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,Россия,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,7/13/2023 01:35:04 PM,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,Россия,347,Москва,1,Центральный,0,Инженер связи,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,12/16/2022 11:54:22 AM,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,NaN,...,Россия,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме..."
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,12/26/2022 10:11:42 PM,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,Россия,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[]
171137,hh,fcc1a67f-a63b-4194-f288-aa7173e0830e,76596362,2/3/2023 02:10:00 PM,0,Кладовщица / Кладовщик,Обязанности: знание методов ведения и учета с...,40000,45000,RUR,...,Россия,464,Рязанская область,1,Центральный,52,Кладовщик,[],[],[]
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,4/27/2023 06:38:18 AM,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,Казахстан,26,Алматы,0,NaN,40,Другое,[],[],[]


In [ ]:
df['role_name_normalized'] = (
    df['name']
    .str.lower()
    .str.replace(r'[^\w\s]', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)
df.head()

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,role_name_normalized
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,9/10/2023 03:35:02 PM,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],инженер по пусконаладке станков сервисный инже...
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,9/9/2023 03:40:03 PM,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[],инженер 2 категории
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,8/31/2023 01:20:03 PM,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],начальник кабинета автоматизированных систем у...
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,3/18/2023 01:20:03 PM,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[],senior 1с программист
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,7/13/2023 01:35:04 PM,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,347,Москва,1,Центральный,0,Инженер связи,[],[],[],инженер связи


вместо нанок впишем нолики или сделаем пустые строки

In [ ]:
df['salary_from'] = df['salary_from'].fillna(0)
df['salary_to'] = df['salary_to'].fillna(0)
df['region_name'] = df['region_name'].fillna('')
df['experience'] = df['experience'].fillna('')
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,role_name_normalized
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,9/10/2023 03:35:02 PM,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],инженер по пусконаладке станков сервисный инже...
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,9/9/2023 03:40:03 PM,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[],инженер 2 категории
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,8/31/2023 01:20:03 PM,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],начальник кабинета автоматизированных систем у...
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,3/18/2023 01:20:03 PM,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[],senior 1с программист
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,7/13/2023 01:35:04 PM,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,347,Москва,1,Центральный,0,Инженер связи,[],[],[],инженер связи
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,12/16/2022 11:54:22 AM,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,NaN,...,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",руководитель астрологического направления для b2b
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,12/26/2022 10:11:42 PM,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],мерчендайзер стационарный
171137,hh,fcc1a67f-a63b-4194-f288-aa7173e0830e,76596362,2/3/2023 02:10:00 PM,0,Кладовщица / Кладовщик,Обязанности: знание методов ведения и учета с...,40000,45000,RUR,...,464,Рязанская область,1,Центральный,52,Кладовщик,[],[],[],кладовщица кладовщик
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,4/27/2023 06:38:18 AM,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,26,Алматы,0,NaN,40,Другое,[],[],[],печатник


создадим ключ чтобы искать дубликаты ну и тут же удалим их с его помощью


In [ ]:
strict_dups = df.duplicated(
    subset=['role_name_normalized'],
    keep='first'
)
strict_dups

,0
0,False
1,False
2,False
3,False
4,False
...,...
171135,False
171136,True
171137,True
171138,True


In [ ]:
df = df.drop_duplicates(subset=['role_name_normalized', 'date_published', 'employer_name', 'region_name'], keep='first')
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,role_name_normalized
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,9/10/2023 03:35:02 PM,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],инженер по пусконаладке станков сервисный инже...
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,9/9/2023 03:40:03 PM,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[],инженер 2 категории
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,8/31/2023 01:20:03 PM,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],начальник кабинета автоматизированных систем у...
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,3/18/2023 01:20:03 PM,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[],senior 1с программист
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,7/13/2023 01:35:04 PM,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,347,Москва,1,Центральный,0,Инженер связи,[],[],[],инженер связи
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,12/16/2022 11:54:22 AM,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,NaN,...,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",руководитель астрологического направления для b2b
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,12/26/2022 10:11:42 PM,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],мерчендайзер стационарный
171137,hh,fcc1a67f-a63b-4194-f288-aa7173e0830e,76596362,2/3/2023 02:10:00 PM,0,Кладовщица / Кладовщик,Обязанности: знание методов ведения и учета с...,40000,45000,RUR,...,464,Рязанская область,1,Центральный,52,Кладовщик,[],[],[],кладовщица кладовщик
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,4/27/2023 06:38:18 AM,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,26,Алматы,0,NaN,40,Другое,[],[],[],печатник


при помощи ключа, который создается из нормализованного названия, зарплаты, региона и опыта и позволяет находить точные дубликаты, мы почистили точные дубликаты.



пы. сы. Точные дубликаты — это полностью идентичные записи, которые совпадают по всем ключевым параметрам.

сейчас перейдем к почти дубликатам. тоже поясню что это.
Почти дубликаты (частичные дубликаты) — это записи с незначительными различиями в одном-двух параметрах:

Типичные случаи:

Небольшая разница в зарплате (50 000-70 000 и 55 000-72 000)

Разные формулировки названия ("Менеджер по продажам" и "Менеджер продаж")

Одинаковые вакансии от разных HR-агентств

надеюсь норм, если поясню в коде че и как

группируем по нормализированному названию  и региону

In [ ]:
grouped = df.groupby(['role_name_normalized', 'region_name'])
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,role_name_normalized
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,9/10/2023 03:35:02 PM,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],инженер по пусконаладке станков сервисный инже...
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,9/9/2023 03:40:03 PM,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[],инженер 2 категории
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,8/31/2023 01:20:03 PM,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],начальник кабинета автоматизированных систем у...
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,3/18/2023 01:20:03 PM,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[],senior 1с программист
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,7/13/2023 01:35:04 PM,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,347,Москва,1,Центральный,0,Инженер связи,[],[],[],инженер связи
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,12/16/2022 11:54:22 AM,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,NaN,...,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",руководитель астрологического направления для b2b
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,12/26/2022 10:11:42 PM,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],мерчендайзер стационарный
171137,hh,fcc1a67f-a63b-4194-f288-aa7173e0830e,76596362,2/3/2023 02:10:00 PM,0,Кладовщица / Кладовщик,Обязанности: знание методов ведения и учета с...,40000,45000,RUR,...,464,Рязанская область,1,Центральный,52,Кладовщик,[],[],[],кладовщица кладовщик
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,4/27/2023 06:38:18 AM,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,26,Алматы,0,NaN,40,Другое,[],[],[],печатник


ЗП преобразовываем в нампи эррей, чтобы было быстрее сравнивать. будем прверять разницу в зп, если меньше 10 п.п., то дубликат

In [ ]:
duplicates_to_drop = []

for (name, region), group in grouped:
    if len(group) > 1:
        salaries_from = group['salary_from'].values
        salaries_to = group['salary_to'].values
        indices = group.index.values


        for i in range(len(group)):
            for j in range(i+1, len(group)):
                if (abs(salaries_from[i] - salaries_from[j]) < 0.1 * salaries_from[i] and
                    abs(salaries_to[i] - salaries_to[j]) < 0.1 * salaries_to[i]):
                    duplicates_to_drop.append(indices[j])
duplicates_to_drop

[np.int64(92240),
 np.int64(41100),
 np.int64(157922),
 np.int64(102691),
 np.int64(11255),
 np.int64(146990),
 np.int64(160077),
 np.int64(146990),
 np.int64(160077),
 np.int64(116210),
 np.int64(160077),
 np.int64(136730),
 np.int64(110420),
 np.int64(149277),
 np.int64(162986),
 np.int64(123041),
 np.int64(27222),
 np.int64(45733),
 np.int64(85030),
 np.int64(78358),
 np.int64(56506),
 np.int64(165374),
 np.int64(85388),
 np.int64(153369),
 np.int64(100565),
 np.int64(39656),
 np.int64(61474),
 np.int64(150441),
 np.int64(126602),
 np.int64(55492),
 np.int64(131968),
 np.int64(148927),
 np.int64(148927),
 np.int64(91911),
 np.int64(51926),
 np.int64(80718),
 np.int64(124223),
 np.int64(138575),
 np.int64(143817),
 np.int64(80718),
 np.int64(124223),
 np.int64(138575),
 np.int64(143817),
 np.int64(124223),
 np.int64(138575),
 np.int64(143817),
 np.int64(138575),
 np.int64(143817),
 np.int64(143817),
 np.int64(34085),
 np.int64(155176),
 np.int64(162688),
 np.int64(166653),
 np.int64(

получили индексы дубликатов, теперь сносим их

In [ ]:
df = df[~df.index.isin(duplicates_to_drop)]
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,role_name_normalized
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,9/10/2023 03:35:02 PM,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],инженер по пусконаладке станков сервисный инже...
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,9/9/2023 03:40:03 PM,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[],инженер 2 категории
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,8/31/2023 01:20:03 PM,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],начальник кабинета автоматизированных систем у...
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,3/18/2023 01:20:03 PM,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[],senior 1с программист
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,7/13/2023 01:35:04 PM,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,347,Москва,1,Центральный,0,Инженер связи,[],[],[],инженер связи
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,4/10/2023 02:23:46 PM,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUR,...,347,Москва,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[],кладовщик грузчик работник склада
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,12/16/2022 11:54:22 AM,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,NaN,...,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",руководитель астрологического направления для b2b
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,12/26/2022 10:11:42 PM,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],мерчендайзер стационарный
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,4/27/2023 06:38:18 AM,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,26,Алматы,0,NaN,40,Другое,[],[],[],печатник


убираем ненужные теперь столбцы

In [ ]:
df = df.drop(columns=['role_name_normalized'])

df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,country_name,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,9/10/2023 03:35:02 PM,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,Россия,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,9/9/2023 03:40:03 PM,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,Россия,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,8/31/2023 01:20:03 PM,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,Россия,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,3/18/2023 01:20:03 PM,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,Россия,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,7/13/2023 01:35:04 PM,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,Россия,347,Москва,1,Центральный,0,Инженер связи,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,4/10/2023 02:23:46 PM,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUR,...,Россия,347,Москва,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[]
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,12/16/2022 11:54:22 AM,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,NaN,...,Россия,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме..."
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,12/26/2022 10:11:42 PM,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,Россия,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[]
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,4/27/2023 06:38:18 AM,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,Казахстан,26,Алматы,0,NaN,40,Другое,[],[],[]


теперь с датами

In [ ]:
original_dates = df['date_published'].copy()
null_before = df['date_published'].isna().sum()
total_records = len(df)
original_dates

,date_published
0,9/10/2023 03:35:02 PM
1,9/9/2023 03:40:03 PM
2,8/31/2023 01:20:03 PM
3,3/18/2023 01:20:03 PM
4,7/13/2023 01:35:04 PM
...,...
171134,4/10/2023 02:23:46 PM
171135,12/16/2022 11:54:22 AM
171136,12/26/2022 10:11:42 PM
171138,4/27/2023 06:38:18 AM


при помощи автоматического определения проверим все и посмотрим че как

In [ ]:
df['date_published'] = pd.to_datetime(
    df['date_published'],
    dayfirst=True,
    yearfirst=False,
    errors='coerce',
    exact=False,
    utc=False
)
df['date_published']

<ipython-input-423-b848b7cf63ab>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_published'] = pd.to_datetime(


,date_published
0,2023-10-09 15:35:02
1,2023-09-09 15:40:03
2,2023-08-31 13:20:03
3,2023-03-18 13:20:03
4,2023-07-13 13:35:04
...,...
171134,2023-10-04 14:23:46
171135,2022-12-16 11:54:22
171136,2022-12-26 22:11:42
171138,2023-04-27 06:38:18


теперь посмотрим все ли ок



те проверим сколько нормальных

создадим для этого маску

In [ ]:
failed_mask = df['date_published'].isna() & original_dates.notna()
failed_mask

,date_published
0,False
1,False
2,False
3,False
4,False
...,...
171134,False
171135,False
171136,False
171138,False


In [ ]:
print(failed_mask.sum())

0


видим что проблемных мест не осталось

но если бы вдруг они стались надо было бы обработать ручками

In [ ]:
if failed_mask.any():


    formats_priority = [
        ('%d.%m.%Y', '15.12.2023'),
        ('%Y-%m-%d', '2023-12-15'),
        ('%d/%m/%Y', '15/12/2023'),
        ('%d-%m-%Y', '15-12-2023'),
        ('%Y.%m.%d', '2023.12.15'),
        ('%d %b %Y', '15 Dec 2023'),
        ('%d %B %Y', '15 December 2023'),
        ('%b %d, %Y', 'Dec 15, 2023'),
        ('%B %d, %Y', 'December 15, 2023'),
        ('%Y%m%d', '20231215'),
        ('%m/%d/%Y', '12/15/2023'),
        ('%d-%b-%y', '15-Dec-23'),
        ('%d.%m.%y', '15.12.23')
    ]

    for fmt, example in formats_priority:
        current_problems = failed_mask & df['date_published'].isna()
        if not current_problems.any():
            break

        temp_dates = pd.to_datetime(
            original_dates[current_problems],
            format=fmt,
            errors='coerce'
        )

        updated_count = temp_dates.notna().sum()
        if updated_count > 0:
            df.loc[current_problems, 'date_published'] = temp_dates
            failed_mask = df['date_published'].isna() & original_dates.notna()
            print(f"Формат '{example}': обработано {updated_count:,}")

сохраним информацию об ошибках

In [ ]:
final_failed = failed_mask.sum()
success_total = total_records - final_failed - null_before

print(f'успешно: {success_total:,} ({success_total/total_records:.1%})')
print(f'не удалось обработать: {final_failed:,}')
print(f'исходных пропусков:    {null_before:,}')

if final_failed > 0:
    df['date_parse_error'] = failed_mask

успешно: 149,810 (100.0%)
не удалось обработать: 0
исходных пропусков:    0


преобразовываем дататайм в строки

In [ ]:
if pd.api.types.is_datetime64_any_dtype(df['date_published']):
    df.loc[df['date_published'].notna(), 'date_published'] = (
        df.loc[df['date_published'].notna(), 'date_published']
        .dt.strftime('%Y-%m-%d')
    )

df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,country_name,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,Россия,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,Россия,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,Россия,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,Россия,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,Россия,347,Москва,1,Центральный,0,Инженер связи,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUR,...,Россия,347,Москва,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[]
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,NaN,...,Россия,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме..."
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,Россия,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[]
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,Казахстан,26,Алматы,0,NaN,40,Другое,[],[],[]


тепрь перейдем к зарплатам

посмотрим какие вообще валюты представлены у нас

In [ ]:
unique_vals = df['salary_currency'].unique()
print(unique_vals)

['RUR' nan 'USD' 'BYR' 'KZT' 'UZS' 'KGS' 'EUR' 'GEL' 'AZN']


заметим что беларусские рубли старые, те до 2016, надо исправить

тепрь чтобы конвертировать зарплаты, надо полуить актуальные курсы ЦБ

получим данные с сервера ЦБ. Для этого отправляем GET-запрос к API ЦБ, который возвращает XML-файл с текущими курсами валют.

In [ ]:
cbr_response = requests.get('https://www.cbr.ru/scripts/XML_daily.asp')


прверяем был ли запрос успешным, должно вывести 200

In [ ]:
cbr_response.raise_for_status()
cbr_response

<Response [200]>

да, все классно

тепрь спарсим хмл

In [ ]:
root = ET.fromstring(cbr_response.content)
cbr_date = root.attrib['Date']
cbr_date

'30.05.2025'

создаем словарь курсов, где и одинаковая ключевая валюта

In [ ]:
exchange_rates = {'RUB': 1.0, 'RUR': 1.0}
for valute in root.findall('Valute'):
    char_code = valute.find('CharCode').text
    nominal = int(valute.find('Nominal').text)
    value = float(valute.find('Value').text.replace(',', '.'))
    exchange_rates[char_code] = value / nominal

exchange_rates

{'RUB': 1.0,
 'RUR': 1.0,
 'AUD': 50.4029,
 'AZN': 46.1747,
 'GBP': 105.7433,
 'AMD': 0.20433900000000002,
 'BYN': 26.1936,
 'BGN': 45.4207,
 'BRL': 13.7876,
 'HUF': 0.218819,
 'VND': 0.00314466,
 'HKD': 10.0316,
 'GEL': 28.702,
 'DKK': 11.9101,
 'AED': 21.3743,
 'USD': 78.497,
 'EUR': 88.9481,
 'EGP': 1.57799,
 'INR': 0.9181189999999999,
 'IDR': 0.004829099999999999,
 'KZT': 0.153117,
 'CAD': 56.7749,
 'QAR': 21.5651,
 'KGS': 0.897621,
 'CNY': 10.894,
 'MDL': 4.54328,
 'NZD': 46.6233,
 'NOK': 7.7285200000000005,
 'PLN': 20.8835,
 'RON': 17.4733,
 'XDR': 106.4977,
 'SGD': 60.6764,
 'TJS': 7.84029,
 'THB': 2.39218,
 'TRY': 2.01192,
 'TMT': 22.4277,
 'UZS': 0.0061177,
 'UAH': 1.8902,
 'CZK': 3.55544,
 'SEK': 8.16987,
 'CHF': 94.9752,
 'RSD': 0.7532800000000001,
 'ZAR': 4.36993,
 'KRW': 0.0571053,
 'JPY': 0.538758}

теперь заменим пропуски и нормализуем данные

In [ ]:
df['salary_currency'] = (
    df['salary_currency']
    .fillna('RUB')
    .astype(str)
    .str.upper()
    .str.strip()
    .replace({
        'BYR': 'BYN',
        '': 'RUB',
        'РУБ': 'RUB',
        'РУБЛЬ': 'RUB',
        'ДОЛЛАР': 'USD',
        'ЕВРО': 'EUR',
        'ГРИВНА': 'UAH',
        'ТЕНГЕ': 'KZT'
    })
)
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,country_name,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,Россия,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,Россия,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,Россия,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,Россия,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,Россия,347,Москва,1,Центральный,0,Инженер связи,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUR,...,Россия,347,Москва,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[]
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,Россия,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме..."
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,Россия,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[]
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,Казахстан,26,Алматы,0,NaN,40,Другое,[],[],[]


In [ ]:
df['original_currency'] = df['salary_currency']
df['original_salary_from'] = df['salary_from']
df['original_salary_to'] = df['salary_to']
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,original_currency,original_salary_from,original_salary_to
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],RUR,80000,100000
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,5,Уральский,0,Инженер (2 категории),[],[],[],RUR,60000,120000
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],RUR,100000,100000
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,1,Центральный,0,Senior 1С программист,[],[],[],RUR,200000,200000
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,1,Центральный,0,Инженер связи,[],[],[],RUR,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUR,...,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[],RUR,45000,50000
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",RUB,0,0
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],RUR,2000,0
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,270000,0,KZT,...,0,NaN,40,Другое,[],[],[],KZT,270000,0


теперь конвертируем зарплаты в рубли

сперва конвертируем salary_from

In [ ]:
df['salary_from'] = df.apply(
    lambda row: round(row['salary_from'] * exchange_rates.get(row['salary_currency'], 1.0))
    if pd.notna(row['salary_from']) and row['salary_from'] != 0 else 0,
    axis=1
)
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,original_currency,original_salary_from,original_salary_to
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],RUR,80000,100000
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,5,Уральский,0,Инженер (2 категории),[],[],[],RUR,60000,120000
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],RUR,100000,100000
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,1,Центральный,0,Senior 1С программист,[],[],[],RUR,200000,200000
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,1,Центральный,0,Инженер связи,[],[],[],RUR,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUR,...,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[],RUR,45000,50000
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",RUB,0,0
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],RUR,2000,0
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,KZT,...,0,NaN,40,Другое,[],[],[],KZT,270000,0


теперь salary_to

In [ ]:
df['salary_to'] = df.apply(
    lambda row: round(row['salary_to'] * exchange_rates.get(row['salary_currency'], 1.0))
    if pd.notna(row['salary_to']) and row['salary_to'] != 0 else 0,
    axis=1
)
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,original_currency,original_salary_from,original_salary_to
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUR,...,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],RUR,80000,100000
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUR,...,5,Уральский,0,Инженер (2 категории),[],[],[],RUR,60000,120000
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUR,...,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],RUR,100000,100000
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUR,...,1,Центральный,0,Senior 1С программист,[],[],[],RUR,200000,200000
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUR,...,1,Центральный,0,Инженер связи,[],[],[],RUR,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUR,...,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[],RUR,45000,50000
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",RUB,0,0
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUR,...,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],RUR,2000,0
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,KZT,...,0,NaN,40,Другое,[],[],[],KZT,270000,0


соответственно обновляем валюту после конвертации

In [ ]:
df['salary_currency'] = 'RUB'
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,original_currency,original_salary_from,original_salary_to
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],RUR,80000,100000
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,5,Уральский,0,Инженер (2 категории),[],[],[],RUR,60000,120000
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],RUR,100000,100000
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUB,...,1,Центральный,0,Senior 1С программист,[],[],[],RUR,200000,200000
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUB,...,1,Центральный,0,Инженер связи,[],[],[],RUR,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUB,...,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[],RUR,45000,50000
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",RUB,0,0
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUB,...,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],RUR,2000,0
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,RUB,...,0,NaN,40,Другое,[],[],[],KZT,270000,0


для удобства добавим дату обработки


но наверное потом уберу

In [ ]:
df['conversion_date'] = datetime.now().strftime('%Y-%m-%d')

df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,original_currency,original_salary_from,original_salary_to,conversion_date
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],RUR,80000,100000,2025-05-29
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,Уральский,0,Инженер (2 категории),[],[],[],RUR,60000,120000,2025-05-29
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],RUR,100000,100000,2025-05-29
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUB,...,Центральный,0,Senior 1С программист,[],[],[],RUR,200000,200000,2025-05-29
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUB,...,Центральный,0,Инженер связи,[],[],[],RUR,0,0,2025-05-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUB,...,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[],RUR,45000,50000,2025-05-29
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...",RUB,0,0,2025-05-29
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUB,...,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],RUR,2000,0,2025-05-29
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,RUB,...,NaN,40,Другое,[],[],[],KZT,270000,0,2025-05-29


In [ ]:
print(df.columns.tolist())

['data_source', 'v_id', 'id', 'date_published', 'is_open', 'name', 'description', 'salary_from', 'salary_to', 'salary_currency', 'experience', 'employer_name', 'employer_type_label', 'employer_industry_id', 'employer_industry_name', 'employer_department_name', 'accept_kids', 'accept_handicapped', 'schedule_label', 'employment_label', 'country_id', 'country_name', 'region_id', 'region_name', 'region_district_id', 'region_district_name', 'role_id', 'role_name', 'skills', 'hard_skills', 'soft_skills', 'original_currency', 'original_salary_from', 'original_salary_to', 'conversion_date']


все таки решила убрать, зачем таскать лишнее

In [ ]:
df.drop(columns=['original_salary_from', 'conversion_date', 'original_currency', 'original_salary_to'], inplace=True)

df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,country_name,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,Россия,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,Россия,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,Россия,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUB,...,Россия,347,Москва,1,Центральный,0,Senior 1С программист,[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUB,...,Россия,347,Москва,1,Центральный,0,Инженер связи,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUB,...,Россия,347,Москва,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[]
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,Россия,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме..."
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUB,...,Россия,347,Москва,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[]
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,RUB,...,Казахстан,26,Алматы,0,NaN,40,Другое,[],[],[]


In [ ]:
unique_skills = df['skills'].unique()
print(unique_skills)

['[]'
 '[Управление грузовым автомобилем, Управление легковым автомобилями и небольшими грузовиками]'
 '[Базовая инструментальная грамотность, Креативное мышление, Организаторские навыки, Целеустремленность и работоспособность]'
 ...
 '[Знание основ в области маркетинга, Знание основ в области маркетинга, SWOT-анализ, Анализ конкурентной среды, Анализ конкурентной среды, Самоорганизованность, Проведение маркетинговых исследований]'
 '[Коммуникативная грамотность, Базовая инструментальная грамотность, Креативное мышление, Публичные выступления и проведение презентаций, Работа в команде, Стрессоустойчивость, Стрессоустойчивость]'
 '[Microsoft Excel, Microsoft Word, Базовая инструментальная грамотность, Коммуникативная грамотность, Организаторские навыки, Базовые знания ПК, Коммуникативная грамотность]']


In [ ]:
required_cols = ['skills', 'soft_skills', 'hard_skills']

посмотри исходную инфу

In [ ]:
print(f"Всего вакансий: {len(df)}")
for col in required_cols:
    cnt = df[col].str.contains(r'\w').sum()
    print(f"Вакансий с {col}: {cnt} ({cnt/len(df):.1%})")

Всего вакансий: 149810
Вакансий с skills: 83696 (55.9%)
Вакансий с soft_skills: 57311 (38.3%)
Вакансий с hard_skills: 71053 (47.4%)


скопируем исходные коллонки на всякий

In [ ]:
required_cols = ['skills', 'soft_skills', 'hard_skills']
for col in required_cols:
    df[f'original_{col}'] = df[col].copy()
    df[col] = df[col].fillna('').astype(str)
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,original_skills,original_soft_skills,original_hard_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,[],[],[],[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,5,Уральский,0,Инженер (2 категории),[],[],[],[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,[],[],[],[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUB,...,1,Центральный,0,Senior 1С программист,[],[],[],[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUB,...,1,Центральный,0,Инженер связи,[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUB,...,1,Центральный,31,Грузчик,[Выполнение производственного технического кон...,[Выполнение производственного технического кон...,[],[Выполнение производственного технического кон...,[],[Выполнение производственного технического кон...
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,2,Северо-Западный,40,Другое,"[Коммуникативная грамотность, Базовая инструме...",[],"[Коммуникативная грамотность, Базовая инструме...","[Коммуникативная грамотность, Базовая инструме...","[Коммуникативная грамотность, Базовая инструме...",[]
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUB,...,1,Центральный,77,Мерчандайзер,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]","[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]"
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,RUB,...,0,NaN,40,Другое,[],[],[],[],[],[]


долго мучалась, сперва сама создавала словарь, потом поняла свою тупость



поэтому попыталась написать функцию для того, чтобы словарь синонимов создавался сам.


для этого сперва собираем все уникальные навыки из колонки, группируем похожие.
когда создаем словарь, вариант который чаще всего упоминается становится каноническим. и самый длинный вариант - основной

In [ ]:
def build_synonym_dicts(df, cols):
    synonym_dicts = {col: defaultdict(list) for col in cols}

    for col in cols:
        all_skills = set()
        for skills_str in df[col]:
            skills = [s.strip().lower() for s in re.split(r'[,/|]', skills_str) if s.strip()]
            all_skills.update(skills)

        skill_groups = defaultdict(list)
        for skill in all_skills:
            base_form = re.sub(r'[^a-zа-яё]', '', skill)
            skill_groups[base_form].append(skill)

        for base_form, variants in skill_groups.items():
            if len(variants) > 1:
                main_variant = max(variants, key=len)
                for variant in variants:
                    if variant != main_variant:
                        synonym_dicts[col][variant] = main_variant

    return synonym_dicts


In [ ]:
synonym_dicts = build_synonym_dicts(df, required_cols)
synonym_dicts


{'skills': defaultdict(list,
             {'steinberg cubase': '[steinberg cubase]',
              'steinberg cubase]': '[steinberg cubase]',
              '[steinberg cubase': '[steinberg cubase]',
              'prometheus': 'prometheus]',
              'позитивное мышление]': '[позитивное мышление]',
              'позитивное мышление': '[позитивное мышление]',
              '[позитивное мышление': '[позитивное мышление]',
              'adobe animate': '[adobe animate',
              'микроконтроллеры и микропроцессоры]': '[микроконтроллеры и микропроцессоры',
              'микроконтроллеры и микропроцессоры': '[микроконтроллеры и микропроцессоры',
              'разработка айдентики бренда]': '[разработка айдентики бренда',
              'разработка айдентики бренда': '[разработка айдентики бренда',
              'бюджетирование]': '[бюджетирование]',
              '[бюджетирование': '[бюджетирование]',
              'бюджетирование': '[бюджетирование]',
              'выполнение

добавим слова от себя

In [ ]:
base_rules = {
    'skills': {
        'кодинг': 'программирование',
        'data analysis': 'анализ данных'
    },
    'soft_skills': {
        'общение': 'коммуникация',
        'командная работа': 'работа в команде'
    },
    'hard_skills': {
        'js': 'javascript',
        'бд': 'sql'
    }
}

In [ ]:
for col in required_cols:
    synonym_dicts[col].update(base_rules.get(col, {}))

скачиваем  стоп слова

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

загрузка русских стоп-слов из NLTK и добавление своих. и объединяем их

In [ ]:
russian_stopwords = set(stopwords.words('russian'))
common_stop_words = russian_stopwords.union({
    'навыки', 'умение', 'знание', 'работа', 'с', 'в', 'на', 'из', 'от', 'до'
})

In [ ]:
stop_words

{'skills': {'в',
  'для',
  'до',
  'за',
  'знание',
  'и',
  'из',
  'или',
  'на',
  'навыки',
  'не',
  'от',
  'по',
  'работа',
  'с',
  'у',
  'умение'},
 'soft_skills': {'в',
  'для',
  'до',
  'за',
  'знание',
  'и',
  'из',
  'или',
  'на',
  'навыки',
  'не',
  'от',
  'по',
  'работа',
  'с',
  'у',
  'умение'},
 'hard_skills': {'в',
  'для',
  'до',
  'за',
  'знание',
  'и',
  'из',
  'или',
  'на',
  'навыки',
  'не',
  'от',
  'по',
  'работа',
  'с',
  'у',
  'умение'}}

запишем функцию для обработки текста



сперва приводим все книжнему регистру

 разделяем навыки и обрабатываем каждый навык

 ну соотвественно синонимы стоп слова все исправляем

In [ ]:
def process_text(text, synonyms, stop_words):
    if not text.strip():
        return ''

    text = text.lower()
    text = re.sub(r'[^\w\s,]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    skills = []
    for part in text.split(','):
        part = part.strip()
        if part:
            skills.extend(re.split(r'[/|]', part))

    processed_skills = []
    for skill in skills:
        skill = skill.strip()
        if not skill:
            continue

        normalized = synonyms.get(skill, skill)

        normalized = ' '.join(
            word for word in normalized.split()
            if word not in stop_words and len(word) > 2
        )

        if normalized and normalized not in processed_skills:
            processed_skills.append(normalized)

    return ', '.join(processed_skills)

обрабатываем каждую колонку

In [ ]:
for col in required_cols:
    df[col] = df[col].apply(
        lambda x: process_text(x, synonym_dicts[col], common_stop_words)
    )

In [ ]:
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,original_skills,original_soft_skills,original_hard_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,,,,[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,5,Уральский,0,Инженер (2 категории),,,,[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,,,,[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUB,...,1,Центральный,0,Senior 1С программист,,,,[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUB,...,1,Центральный,0,Инженер связи,,,,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUB,...,1,Центральный,31,Грузчик,[выполнение производственного технического кон...,[выполнение производственного технического кон...,,[Выполнение производственного технического кон...,[],[Выполнение производственного технического кон...
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,2,Северо-Западный,40,Другое,"[коммуникативная грамотность], [базовая инстру...",,"[коммуникативная грамотность], [базовая инстру...","[Коммуникативная грамотность, Базовая инструме...","[Коммуникативная грамотность, Базовая инструме...",[]
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUB,...,1,Центральный,77,Мерчандайзер,[мерчандайзинг],[мерчандайзинг],,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]"
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,RUB,...,0,NaN,40,Другое,,,,[],[],[]


убираем квадратные скобки

In [ ]:
cols_to_clean = ['skills',	'hard_skills',	'soft_skills']

df[cols_to_clean] = df[cols_to_clean].apply(
    lambda col: col.str.replace(r'[\[\]]', '', regex=True)
)
df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,original_skills,original_soft_skills,original_hard_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,,,,[],[],[]
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,5,Уральский,0,Инженер (2 категории),,,,[],[],[]
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,,,,[],[],[]
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUB,...,1,Центральный,0,Senior 1С программист,,,,[],[],[]
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUB,...,1,Центральный,0,Инженер связи,,,,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUB,...,1,Центральный,31,Грузчик,выполнение производственного технического конт...,выполнение производственного технического конт...,,[Выполнение производственного технического кон...,[],[Выполнение производственного технического кон...
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,2,Северо-Западный,40,Другое,"коммуникативная грамотность, базовая инструмен...",,"коммуникативная грамотность, базовая инструмен...","[Коммуникативная грамотность, Базовая инструме...","[Коммуникативная грамотность, Базовая инструме...",[]
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUB,...,1,Центральный,77,Мерчандайзер,мерчандайзинг,мерчандайзинг,,"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]",[],"[Мерчандайзинг, Мерчандайзинг, Мерчандайзинг]"
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,RUB,...,0,NaN,40,Другое,,,,[],[],[]


удаляем исходные стобцы навыков

In [ ]:
df.drop(columns=['original_skills',	'original_soft_skills',	'original_hard_skills'], inplace=True)

df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,country_name,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,Россия,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,,,
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,Россия,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),,,
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,Россия,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,,,
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUB,...,Россия,347,Москва,1,Центральный,0,Senior 1С программист,,,
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUB,...,Россия,347,Москва,1,Центральный,0,Инженер связи,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUB,...,Россия,347,Москва,1,Центральный,31,Грузчик,выполнение производственного технического конт...,выполнение производственного технического конт...,
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,Россия,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"коммуникативная грамотность, базовая инструмен...",,"коммуникативная грамотность, базовая инструмен..."
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUB,...,Россия,347,Москва,1,Центральный,77,Мерчандайзер,мерчандайзинг,мерчандайзинг,
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,RUB,...,Казахстан,26,Алматы,0,NaN,40,Другое,,,


переходим к категоризации опыта работы. посмортим на нужные столбцы.

In [ ]:
selected_data = df.loc[:, ['name', 'experience']]
selected_data

,name,experience
0,Инженер по пусконаладке станков / Сервисный ин...,1-3
1,Инженер (2 категории),1-3
2,Начальник кабинета автоматизированных систем у...,1-3
3,Senior 1С программист,1-3
4,Инженер связи,1-3
...,...,...
171134,Кладовщик / грузчик / работник склада,0
171135,Руководитель астрологического направления для B2B,1-3
171136,Мерчендайзер Стационарный,0
171138,Печатник,1-3


категоризацию будем проводить через assign и лямбду

In [ ]:
df = df.assign(
    experience_category=lambda x: x['experience'].apply(
        lambda exp: (
            "Без опыта" if pd.isna(exp) or str(exp).lower() in ['нет опыта', 'без опыта', '0']
            else "1-3 года" if str(exp) == "1-3" or (isinstance(exp, (int, float)) and 1 <= float(exp) <= 3)
            else "Более 3 лет"
        )
    )
)

df

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,experience_category
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,,,,1-3 года
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),,,,1-3 года
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,,,,1-3 года
3,superjob,0130559c-07d5-c042-80ca-722d4441df74,1d435c52-964f-11ed-a841-dbfed3997e57,2023-03-18,0,Senior 1С программист,Роль в компании: Доработка нового функционала ...,200000,200000,RUB,...,347,Москва,1,Центральный,0,Senior 1С программист,,,,1-3 года
4,superjob,013f14be-f59f-5d61-4627-60dfb324ea2b,d026f352-4b94-11ed-8804-dbfed3997e57,2023-07-13,0,Инженер связи,Обязанности:. Эксплуатация и настройка внутрен...,0,0,RUB,...,347,Москва,1,Центральный,0,Инженер связи,,,,1-3 года
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171134,hh,fcc133d7-318b-2dec-3743-cb4df8f7768c,79108952,2023-10-04,0,Кладовщик / грузчик / работник склада,Обязанности: - Прием и выдача товара в произ...,45000,50000,RUB,...,347,Москва,1,Центральный,31,Грузчик,выполнение производственного технического конт...,выполнение производственного технического конт...,,Без опыта
171135,hh,fcc14044-f216-3dd5-8521-380f8c43315d,73894723,2022-12-16,0,Руководитель астрологического направления для B2B,Привет! Мы в брендинговом агентстве ENDY ищем...,0,0,RUB,...,475,Санкт-Петербург,2,Северо-Западный,40,Другое,"коммуникативная грамотность, базовая инструмен...",,"коммуникативная грамотность, базовая инструмен...",1-3 года
171136,hh,fcc14f03-01b7-fe76-1746-092bd783db6d,74274777,2022-12-26,0,Мерчендайзер Стационарный,ДатаМаркетинг - крупнейшая компания в сфере FM...,2000,0,RUB,...,347,Москва,1,Центральный,77,Мерчандайзер,мерчандайзинг,мерчандайзинг,,Без опыта
171138,hh,fcc1ba08-bea8-66cc-b9d2-d5f2b3d4cf34,79853966,2023-04-27,0,Печатник,ТОО &quot;ALA Sintez&quot; производит набор пе...,41342,0,RUB,...,26,Алматы,0,NaN,40,Другое,,,,1-3 года


In [ ]:
df.to_csv('hh.csv', index=False, encoding='utf-8-sig')

In [ ]:
from google.colab import files
files.download('hh.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>